In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import regex as re
from jellyfish import jaro_similarity
from tqdm import tqdm

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from time import time
from time import sleep
from time import strftime
from time import gmtime
from selenium.common.exceptions import NoSuchElementException

In [3]:
filename = "covid_vaccine"

In [4]:
path = "../datasets/" + filename + "/" + filename + ".csv"
video_df = pd.read_csv(path).drop("Unnamed: 0", axis=1)

In [5]:
path = "../datasets/" + filename + "/source_links.csv"
sc_df = pd.read_csv(path).drop("Unnamed: 0", axis=1)

---

In [ ]:
# Automated
def get_video_backlinks(video_id_list):
    backlinks = {}
    backlinks["video_id"] = {}
    backlinks["backlinks"] = {}
    
    for i in range(0, len(video_id_list)):
        backlinks["video_id"][i] = video_id_list[i]
    
    driver = webdriver.Firefox()
    driver.get("https://www.thehoth.com/backlinks-checker/")
    
    index = 0
    captcha_passed = False
    
    pbar = tqdm(total=len(video_id_list))
    pbar.set_description("Getting backlinks...")
    start_time = time()
    
    sleep(5)
    
    for video_id in video_id_list:
        # Wait for page to load and locate textbox
        # print("Waiting for frame")
        WebDriverWait(driver, 200).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, "hothtools"))
        )
        
        sleep(1)
        
        WebDriverWait(driver, 200).until_not(
            EC.visibility_of_element_located((By.CLASS_NAME, "hoth-loader__container"))
        )
        
        # Input video link
        # print("Inputting video link")
        notFound = False
        while(not notFound):
            try:
                targeturl = driver.find_element(By.ID, "targeturl")
                notFound = True
            except:
                sleep(10)
                pass
        
        targeturl.click()
        targeturl.send_keys(Keys.CONTROL + "A")
        targeturl.send_keys(Keys.BACKSPACE)
        targeturl.send_keys(f"https://www.youtube.com/watch?v={video_id}")

        # Select 'This Exact URL'
        dropdown = driver.find_element(By.ID, "mode")
        dropdown.click()
        option = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/form/div[2]/select/option[3]")
        option.click()

        # Submit
        submit = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/form/div[2]/button")
        submit.click()

        sleep(5)

        if not captcha_passed:
            try:
                # In case of Captcha page
                # print("Solving CAPTCHA")
                iframes = driver.find_elements(By.TAG_NAME, "iframe")
                cont = False
                for iframe in iframes:
                    try:
                        driver.switch_to.frame(iframe)
                        checkbox = driver.find_element(By.ID, "recaptcha-anchor")
                        checkbox.click()
                        cont = True
                        break
                    except:
                        # print("Not Found")
                        driver.switch_to.default_content()

                if not cont:
                    raise Exception("No CAPTCHA iframe found.")

                # Giving enough time to manually solve CAPTCHA puzzle
                sleep(15)

                driver.switch_to.default_content()
                driver.switch_to.frame("hothtools")
                submit = driver.find_element(
                    By.XPATH, "//*[@id='submit']"
                )
                submit.click()
                captcha_passed = True
                sleep(3)
            except:
                pass
        
        # Get external backlinks
        # print("Getting results")
        driver.switch_to.default_content()
        driver.switch_to.frame("hothtools")
        try:
            # print("Waiting for result frame")
            WebDriverWait(driver, 200).until(
                EC.element_to_be_clickable((By.ID, "targeturl"))
            )

            result = driver.find_element(
                By.XPATH, "/html/body/div[2]/div/div/div[2]/div[2]/div/div/div[1]/div[2]"
            ).text
        except NoSuchElementException:
            result = '0'
        finally:
            backlinks["backlinks"][index] = int(result.replace(',', ''))
            # print(f"--------------------- INDEX {index} COMPLETE ({video_id}: {result}) ---------------------")
            index += 1
            pbar.update(1)
            driver.switch_to.default_content()
    
    pbar.close()
    elapsed_time = time() - start_time
    print('Execution time:', strftime("%H:%M:%S", gmtime(elapsed_time)))
    driver.close()
    
    return backlinks

In [6]:
# Automated
def get_backlinks(df, mode):
    backlinks = {}
    url_list = []
    index = 0
    
    if mode == 'video':
        cols = ['video_id', 'backlinks']
        for col in cols:
            backlinks[col] = {}
            
        video_id_list = df['video_id'].tolist()
        for i in range(len(video_id_list)):
            backlinks['video_id'][i] = video_id_list[i]
        
        cols = [cols[1]]
            
        for i in range(df.shape[0]):
            url_list.append("https://www.youtube.com/watch?v=" + df.iloc[i]['video_id'])
            
    elif mode == 'external':
        cols = ['channel_id', 'channel_name', 'LinkedIn', 'Wiki', 'Website', 'Twitter', 'Facebook']
        for col in cols:
            backlinks[col] = {}
            
        channel_id_list = df['channel_id'].tolist()
        channel_name_list = df['channel_name'].tolist()
        for i in range(len(channel_id_list)):
            backlinks['channel_id'][i] = channel_id_list[i]
            backlinks['channel_name'][i] = channel_name_list[i]
            
        cols = cols[2:7]
        
        for col in cols:
            for i in range(df.shape[0]):
                url_list.append(df.iloc[i][col])
    
    driver = webdriver.Firefox()
    driver.get("https://www.thehoth.com/backlinks-checker/")
    
    captcha_passed = False
    
    pbar = tqdm(total=len(url_list))
    pbar.set_description("Getting backlinks...")
    start_time = time()
    
    for col in cols:
        for i in range(df.shape[0]):
            if url_list[index] is np.nan:
                backlinks[col][i] = 0
                # print(f"--------------------- INDEX {index} COMPLETE ({col}: {i}: {result}) ---------------------")
                index += 1
                pbar.update(1)
                continue
            else:
                # Wait for page to load and locate textbox
                # print("Waiting for frame")
                WebDriverWait(driver, 200).until(
                    EC.frame_to_be_available_and_switch_to_it((By.ID, "hothtools"))
                )

                sleep(1)

                WebDriverWait(driver, 200).until_not(
                    EC.visibility_of_element_located((By.CLASS_NAME, "hoth-loader__container"))
                )

                # Input link
                # print("Inputting link")
                notFound = False
                while(not notFound):
                    try:
                        targeturl = driver.find_element(By.ID, "targeturl")
                        notFound = True
                    except:
                        sleep(10)
                        pass

                targeturl.click()
                targeturl.send_keys(Keys.CONTROL + "A")
                targeturl.send_keys(Keys.BACKSPACE)
                targeturl.send_keys(url_list[index])

                # Select 'This Exact URL'
                dropdown = driver.find_element(By.ID, "mode")
                dropdown.click()
                option = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/form/div[2]/select/option[3]")
                option.click()

                # Submit
                submit = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/form/div[2]/button")
                submit.click()

                sleep(5)

                if not captcha_passed:
                    try:
                        # In case of Captcha page
                        # print("Solving CAPTCHA")
                        iframes = driver.find_elements(By.TAG_NAME, "iframe")
                        cont = False
                        for iframe in iframes:
                            try:
                                driver.switch_to.frame(iframe)
                                checkbox = driver.find_element(By.ID, "recaptcha-anchor")
                                checkbox.click()
                                cont = True
                                break
                            except:
                                # print("Not Found")
                                driver.switch_to.default_content()

                        if not cont:
                            raise Exception("No CAPTCHA iframe found.")

                        # Giving enough time to manually solve CAPTCHA puzzle
                        sleep(15)

                        driver.switch_to.default_content()
                        driver.switch_to.frame("hothtools")
                        submit = driver.find_element(
                            By.XPATH, "//*[@id='submit']"
                        )
                        submit.click()
                        captcha_passed = True
                        sleep(3)
                    except:
                        pass

                # Get external backlinks
                # print("Getting results")
                driver.switch_to.default_content()
                driver.switch_to.frame("hothtools")
                try:
                    # print("Waiting for result frame")
                    WebDriverWait(driver, 200).until(
                        EC.element_to_be_clickable((By.ID, "targeturl"))
                    )

                    result = driver.find_element(
                        By.XPATH, "/html/body/div[2]/div/div/div[2]/div[2]/div/div/div[1]/div[2]"
                    ).text
                except NoSuchElementException:
                    result = '0'
                finally:
                    backlinks[col][i] = int(result.replace(',', ''))
                    # print(f"--------------------- INDEX {index} COMPLETE ({col}: {i}: {result}) ---------------------")
                    index += 1
                    pbar.update(1)
                    driver.switch_to.default_content()

    pbar.close()
    elapsed_time = time() - start_time
    print('Execution time:', strftime("%H:%M:%S", gmtime(elapsed_time)))
    driver.close()
            
    return backlinks

In [7]:
video_backlinks = get_backlinks(video_df, mode='video')

Getting backlinks...: 100%|██████████| 150/150 [19:25<00:00,  7.77s/it]


Execution time: 00:19:25


In [8]:
source_backlinks = get_backlinks(sc_df, mode='external')

Getting backlinks...: 100%|██████████| 530/530 [42:03<00:00,  4.76s/it]


Execution time: 00:42:03


In [13]:
vbl_df = pd.DataFrame.from_dict(video_backlinks)
vbl_df.head()

,video_id,backlinks
0,im3otpqYAiQ,26
1,SkcAZfrYYXM,1
2,7MAlEYqWUTk,4
3,uiwjAj0zfKQ,19
4,jPs4_MeuX7U,2


In [14]:
sbl_df = pd.DataFrame.from_dict(source_backlinks)
sbl_df.head()

,channel_id,channel_name,LinkedIn,Wiki,Website,Twitter,Facebook
0,UCIALMKvObZNtJ6AmdCLP7Lg,Bloomberg Television,751,1237,0,0,30052
1,UC0vn8ISa4LKMunLbzaXLnOQ,GBNews,0,1076,995,12753,9645
2,UCLbidg2ZT49dWrxDkwNKr0g,Doc Schmidt,0,27185,1,0,0
3,UCaZ2jG4qd1vY-Tkwiq9FBlw,Rich Black Guy,1,0,1,10,0
4,UC_gUM8rL-Lrg6O3adPW9K1g,WION,0,820,9070,73,0


In [15]:
path = "../datasets/" + filename + "/video_backlinks.csv"
vbl_df.to_csv(path)

In [16]:
path = "../datasets/" + filename + "/source_backlinks.csv"
sbl_df.to_csv(path)

---

In [19]:
path = "../datasets/" + filename + "/video_backlinks.csv"
vbl_df = pd.read_csv(path).drop("Unnamed: 0", axis=1)
vbl_df["backlinks"].describe()

count     150.000000
mean      114.466667
std       360.553316
min         0.000000
25%         1.000000
50%        10.000000
75%        80.750000
max      3114.000000
Name: backlinks, dtype: float64

In [20]:
path = "../datasets/" + filename + "/source_backlinks.csv"
sbl_df = pd.read_csv(path).drop("Unnamed: 0", axis=1)
sbl_df[['LinkedIn', 'Wiki', 'Website', 'Twitter', 'Facebook']].describe().T

,count,mean,std,min,25%,50%,75%,max
LinkedIn,106.0,3540.981132,2.526643e+04,0.0,0.00,0.0,0.00,192974.0
Wiki,106.0,7511.301887,3.911877e+04,0.0,0.00,0.0,200.25,353421.0
Website,106.0,202524.735849,1.004605e+06,0.0,0.25,961.0,4117.75,6916629.0
Twitter,106.0,17433.169811,9.860762e+04,0.0,0.00,30.5,1252.75,720998.0
Facebook,106.0,2287.896226,1.850099e+04,0.0,0.00,0.0,66.75,188271.0
